In [1]:
import pandas as pd
import pickle as pkl
from constants import *

In [2]:
df = pd.read_excel(r"./Data/output.xlsx", engine='openpyxl')

In [17]:
with open("./gpt35_frames/frames.pkl", "rb") as fp:
    gptframes = pkl.load(fp)

In [18]:
def modify_columns(model_frames, risks):
    risks = [f"{risks[i]}_ranking" for i in range(len(risks))]
    justifications = [f"{risks[i]}_Justifications" for i in range(len(risks))]

    for i in range(len(risks)):
        model_frames[f"df{i}"] = model_frames[f"df{i}"].rename(columns = {"ranks" : risks[i], "Justification" : justifications[i]})


    for i in range(len(risks)):
        stripped_Names = [txt.strip() for txt in model_frames[f"df{i}"]['Use Case Name']]
        model_frames[f"df{i}"]['Use Case Name'] = stripped_Names
    
    return model_frames, risks, justifications


def join_all(model_frames, risks):
    start_df = model_frames['df0']

    i = 1
    while i < len(risks):
        start_df = start_df.merge(model_frames[f"df{i}"], on = ['Use Case ID', 'Use Case Name'])
        i += 1

    return start_df

In [19]:
gptframes,_,_ = modify_columns(gptframes, risk_dimensions_classes)

In [14]:
gptframes = join_all(gptframes, risk_dimensions)

In [15]:
len(gptframes['Use Case ID'].unique())

36

In [20]:
len(gptframes['df0']['Use Case ID'])

58

In [28]:
len(gptframes['df6']['Use Case ID'].unique())

54